In [1]:
# Pacotes
import pandas as pd
from itertools import permutations
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import LinearRegression
from functools import partial,reduce

In [2]:
def all_pairs(df):
    # Pegando Tickets
    tickets = [col for col in df.columns if col != 'Date']
    # Gerando todas as combinaçoes de tickets
    return list(permutations(tickets, 2))

In [3]:
def get_resid(df,pair):
    # pegando primeiro par
    df_train = df[list(pair)]
    # Retirando valores NA
    df_train = df_train.dropna()
    # Dividindo os dados
    x = df_train.iloc[:,1].values.reshape(-1,1)
    y = df_train.iloc[:,0].values.reshape(-1,1)
    # Gerando o modelo
    reg = LinearRegression().fit(x,y)
    # Obtendo previsao
    y_predict = reg.predict(x)
    # obtendo residuos
    return y - y_predict

In [4]:
# confidence interval
def adf_test(resid):
    test_adf = adfuller(resid)
    conf = 1 - test_adf[1]
    if conf > 0.95:
        return True
    else:
        return False

In [5]:
def get_df_pairs_stationary(df,pair):
    # gerando residuos
    resid = get_resid(df,pair)
    # Gerando Teste ADF
    estacionario = adf_test(resid)
    return pd.DataFrame({
    'pair':[pair],
    'estacionario':[estacionario],
    })

In [6]:
def get_df_tickets_cointegration(df):
    # Gerando pares
    pairs = all_pairs(df)
    # Gerando data frames
    list_stationay = list(map(partial(get_df_pairs_stationary, df), pairs))
    return reduce(lambda x,y: pd.concat([x,y]),list_stationay)

In [7]:
# Lendo o dado
df = pd.read_pickle('../data/prices.pickle')
# retirando ASAI3.SA
df = df.drop(columns = 'ASAI3.SA')
# Buscando pares estacionarios
df_stationary = get_df_tickets_cointegration(df)

In [10]:
df_stationary[df_stationary['estacionario'] == True]

,pair,estacionario
0,"(ABEV3.SA, BBAS3.SA)",True
0,"(ABEV3.SA, BBDC3.SA)",True
0,"(ABEV3.SA, BBDC4.SA)",True
0,"(ABEV3.SA, CASH3.SA)",True
0,"(ABEV3.SA, CVCB3.SA)",True
...,...,...
0,"(YDUQ3.SA, MULT3.SA)",True
0,"(YDUQ3.SA, PCAR3.SA)",True
0,"(YDUQ3.SA, RADL3.SA)",True
0,"(YDUQ3.SA, SBSP3.SA)",True
